In [ ]:
import numpy as np
import torch
import torchvision as tvis
import torch.nn as nn
from torchsummary import summary
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
torch.cuda.is_available()

In [ ]:

def get_train_valid_loader(data_dir,
                           batch_size,
                           random_seed,
                           valid_size=0.02,
                           shuffle=True,
                           num_workers=2,
                           pin_memory=True):

    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg


    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor()        ])
    train_transform = transforms.Compose([
            transforms.ToTensor()
        ])

    # load the dataset
    train_dataset = datasets.MNIST(root=data_dir, train=True, 
                download=False, transform=train_transform)

    valid_dataset = datasets.MNIST(root=data_dir, train=True, 
                download=False, transform=valid_transform)

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle == True:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]

    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                    batch_size=batch_size, sampler=train_sampler, 
                    num_workers=num_workers, pin_memory=pin_memory)

    valid_loader = torch.utils.data.DataLoader(valid_dataset, 
                    batch_size=batch_size, sampler=valid_sampler, 
                    num_workers=num_workers, pin_memory=pin_memory)

    return (train_loader, valid_loader)
  

In [ ]:
"""Utility classes for NICE.
"""

import torch
import torch.nn as nn

"""Additive coupling layer.
"""
class Coupling(nn.Module):
    def __init__(self, in_out_dim, mid_dim, hidden, mask_config):
        """Initialize a coupling layer.
        Args:
            in_out_dim: input/output dimensions.
            mid_dim: number of units in a hidden layer.
            hidden: number of hidden layers.
            mask_config: 1 if transform odd units, 0 if transform even units.
        """
        super(Coupling, self).__init__()
        self.mask_config = mask_config

        self.in_block = nn.Sequential(
            nn.Linear(in_out_dim//2, mid_dim),
            nn.ReLU())
        self.mid_block = nn.ModuleList([
            nn.Sequential(
                nn.Linear(mid_dim, mid_dim),
                nn.ReLU()) for _ in range(hidden - 1)])
        self.out_block = nn.Linear(mid_dim, in_out_dim//2)

    def forward(self, x, reverse=False):
        """Forward pass.
        Args:
            x: input tensor.
            reverse: True in inference mode, False in sampling mode.
        Returns:
            transformed tensor.
        """
        [B, W] = list(x.size())
        x = x.reshape((B, W//2, 2))
        if self.mask_config:
            on, off = x[:, :, 0], x[:, :, 1]
        else:
            off, on = x[:, :, 0], x[:, :, 1]

        off_ = self.in_block(off)
        for i in range(len(self.mid_block)):
            off_ = self.mid_block[i](off_)
        shift = self.out_block(off_)
        if reverse:
            on = on - shift
        else:
            on = on + shift

        if self.mask_config:
            x = torch.stack((on, off), dim=2)
        else:
            x = torch.stack((off, on), dim=2)
        return x.reshape((B, W))

"""Log-scaling layer.
"""
class Scaling(nn.Module):
    def __init__(self, dim):
        """Initialize a (log-)scaling layer.
        Args:
            dim: input/output dimensions.
        """
        super(Scaling, self).__init__()
        self.scale = nn.Parameter(
            torch.zeros((1, dim)), requires_grad=True)

    def forward(self, x, reverse=False):
        """Forward pass.
        Args:
            x: input tensor.
            reverse: True in inference mode, False in sampling mode.
        Returns:
            transformed tensor and log-determinant of Jacobian.
        """
        log_det_J = torch.sum(self.scale)
        if reverse:
            x = x * torch.exp(-self.scale)
        else:
            x = x * torch.exp(self.scale)
        return x, log_det_J

"""NICE main model.
"""
class NICE(nn.Module):
    def __init__(self, prior, coupling, 
        in_out_dim, mid_dim, hidden, mask_config):
        """Initialize a NICE.
        Args:
            prior: prior distribution over latent space Z.
            coupling: number of coupling layers.
            in_out_dim: input/output dimensions.
            mid_dim: number of units in a hidden layer.
            hidden: number of hidden layers.
            mask_config: 1 if transform odd units, 0 if transform even units.
        """
        super(NICE, self).__init__()
        self.prior = prior
        self.in_out_dim = in_out_dim

        self.coupling = nn.ModuleList([
            Coupling(in_out_dim=in_out_dim, 
                     mid_dim=mid_dim, 
                     hidden=hidden, 
                     mask_config=(mask_config+i)%2) \
            for i in range(coupling)])
        self.scaling = Scaling(in_out_dim)

    def g(self, z):
        """Transformation g: Z -> X (inverse of f).
        Args:
            z: tensor in latent space Z.
        Returns:
            transformed tensor in data space X.
        """
        x, _ = self.scaling(z, reverse=True)
        for i in reversed(range(len(self.coupling))):
            x = self.coupling[i](x, reverse=True)
        return x

    def f(self, x):
        """Transformation f: X -> Z (inverse of g).
        Args:
            x: tensor in data space X.
        Returns:
            transformed tensor in latent space Z.
        """
        for i in range(len(self.coupling)):
            x = self.coupling[i](x)
        return self.scaling(x)

    def log_prob(self, x):
        """Computes data log-likelihood.
        (See Section 3.3 in the NICE paper.)
        Args:
            x: input minibatch.
        Returns:
            log-likelihood of input.
        """
        z, log_det_J = self.f(x)
        log_ll = torch.sum(self.prior.log_prob(z), dim=1)
        return log_ll + log_det_J

    def sample(self, size):
        """Generates samples.
        Args:
            size: number of samples to generate.
        Returns:
            samples from the data space X.
        """
        z = self.prior.sample((size, self.in_out_dim)).cuda()
        return self.g(z)

    def forward(self, x):
        """Forward pass.
        Args:
            x: input minibatch.
        Returns:
            log-likelihood of input.
        """
        return self.log_prob(x)

import torch
import torch.nn.functional as F

def dequantize(x, dataset):
    '''Dequantize data.
    Add noise sampled from Uniform(0, 1) to each pixel (in [0, 255]).
    Args:
        x: input tensor.
        reverse: True in inference mode, False in training mode.
    Returns:
        dequantized data.
    '''
    noise = torch.distributions.Uniform(0., 1.).sample(x.size())
    return (x * 255. + noise) / 256.

def prepare_data(x, dataset, zca=None, mean=None, reverse=False):
    """Prepares data for NICE.
    In training mode, flatten and dequantize the input.
    In inference mode, reshape tensor into image size.
    Args:
        x: input minibatch.
        dataset: name of dataset.
        zca: ZCA whitening transformation matrix.
        mean: center of original dataset.
        reverse: True if in inference mode, False if in training mode.
    Returns:
        transformed data.
    """
    if reverse:
        assert len(list(x.size())) == 2
        [B, W] = list(x.size())

        if dataset in ['mnist', 'fashion-mnist']:
            assert W == 1 * 28 * 28
            x += mean
            x = x.reshape((B, 1, 28, 28))
        elif dataset in ['svhn', 'cifar10']:
            assert W == 3 * 32 * 32
            x = torch.matmul(x, zca.inverse()) + mean
            x = x.reshape((B, 3, 32, 32))
    else:
        assert len(list(x.size())) == 4
        [B, C, H, W] = list(x.size())

        if dataset in ['mnist', 'fashion-mnist']:
            assert [C, H, W] == [1, 28, 28]
        elif dataset in ['svhn', 'cifar10']:
            assert [C, H, W] == [3, 32, 32]

        x = dequantize(x, dataset)
        x = x.reshape((B, C*H*W))

        if dataset in ['mnist', 'fashion-mnist']:
            x -= mean
        elif dataset in ['svhn', 'cifar10']:
            x = torch.matmul((x - mean), zca)
    return x

"""Standard logistic distribution.
"""
class StandardLogistic(torch.distributions.Distribution):
    def __init__(self):
        super(StandardLogistic, self).__init__()

    def log_prob(self, x):
        """Computes data log-likelihood.
        Args:
            x: input tensor.
        Returns:
            log-likelihood.
        """
        return -(F.softplus(x) + F.softplus(-x))

    def sample(self, size):
        """Samples from the distribution.
        Args:
            size: number of samples to generate.
        Returns:
            samples.
        """
        z = torch.distributions.Uniform(0., 1.).sample(size).cuda()
        return torch.log(z) - torch.log(1. - z)

In [ ]:
import torch, torchvision
import numpy as np



device = torch.device("cuda:0")
data_dir = "./gdrive/My Drive/coding_projects/data"

# model hyperparameters
dataset = "mnist"
batch_size = 200
latent = 512
max_iter = 5000
decay = 0.999
momentum=0.9
lr = 1e-3
sample_size = 64
coupling = 4
mask_config = 1
latent = 'normal'


zca = None
mean = None
mean = torch.load("./gdrive/My Drive/coding_projects/data/MNIST/mnist_mean.pt")
(full_dim, mid_dim, hidden) = (1 * 28 * 28, 1000, 5)


if latent == 'normal':
    prior = torch.distributions.Normal(
        torch.tensor(0.).to(device), torch.tensor(1.).to(device))
elif latent == 'logistic':
    prior = StandardLogistic()

trainloader, vl_load = get_train_valid_loader(data_dir,batch_size=batch_size, num_workers=2, random_seed=0)

flow = NICE(prior=prior, 
            coupling=coupling, 
            in_out_dim=full_dim, 
            mid_dim=mid_dim, 
            hidden=hidden, 
            mask_config=mask_config).to(device)
optimizer = torch.optim.Adam(
    flow.parameters(), lr=lr, betas=(momentum, decay), eps=1e-4)

total_iter = 0
train = True
running_loss = 0

while train:
    for _, data in enumerate(trainloader, 1):
        flow.train()    # set to training mode
        if total_iter == max_iter:
            train = False
            break

        total_iter += 1
        optimizer.zero_grad()    # clear gradient tensors

        inputs, _ = data
        inputs = prepare_data(
            inputs, dataset, zca=zca, mean=mean).to(device)

        # log-likelihood of input minibatch
        loss = -flow(inputs).mean()
        running_loss += float(loss)

        # backprop and update parameters
        loss.backward()
        optimizer.step()

        if total_iter % 1000 == 0:
            mean_loss = running_loss / 1000
            bit_per_dim = (mean_loss + np.log(256.) * full_dim) \
                        / (full_dim * np.log(2.))
            print('iter %s:' % total_iter, 
                'loss = %.3f' % mean_loss, 
                'bits/dim = %.3f' % bit_per_dim)
            running_loss = 0.0

            flow.eval()        # set to inference mode
with torch.no_grad():
    z, _ = flow.f(inputs)
    reconst = flow.g(z).cpu()
    reconst = prepare_data(
        reconst, dataset, zca=zca, mean=mean, reverse=True)
    samples = flow.sample(sample_size).cpu()
    samples = prepare_data(
        samples, dataset, zca=zca, mean=mean, reverse=True)
                



In [ ]:
import matplotlib.pyplot as plt

In [ ]:
index_ = 0
plt.figure()
plt.imshow(inputs[index_].cpu().detach().numpy().reshape(28,28))
plt.figure()
plt.imshow(reconst[index_].cpu().detach().numpy().reshape(28,28))
plt.figure()
plt.imshow(samples[index_].cpu().detach().numpy().reshape(28,28))

In [ ]:
flow.parameters()